In [2]:
import geopandas as gpd
import pandas as pd
import os
import numpy as np

In [3]:
# show all columns
pd.options.display.max_columns = None

## Join Bike volume data to links

In [3]:
# read in links csv
links = gpd.read_file(r".\Data\links.csv")

# read in links shapefile
links_shp = gpd.read_file(r".\Data\links.shp")
print(links.shape)
print(links_shp.shape)


(137281, 34)
(137281, 38)


In [4]:
# read in bike volume
bike_volume = pd.read_csv(r".\Data\bike_vol.csv")

#should be double the amount of links for both directions
print(bike_volume.shape)

# fill bike volume NAs with -1
bike_volume['bike_vol'] = bike_volume['bike_vol'].fillna(0)

(274562, 3)


In [5]:
# Create key to use for joining to links
bike_volume['key'] = np.where(bike_volume['from_node'].astype(int) < bike_volume['to_node'].astype(int), 
                              bike_volume['from_node'].astype(str) + "_"+ bike_volume['to_node'].astype(str), 
                              bike_volume['to_node'].astype(str) + "_"+ bike_volume['from_node'].astype(str))

# Create directional keys
bike_volume['ft_key'] = bike_volume['from_node'].astype(str) + "_"+ bike_volume['to_node'].astype(str)
bike_volume['tf_key'] = bike_volume['to_node'].astype(str) + "_"+ bike_volume['from_node'].astype(str)

bike_volume.head(15)

,from_node,to_node,bike_vol,key,ft_key,tf_key
0,0,732,0.000000,0_732,0_732,732_0
1,0,2638,0.000000,0_2638,0_2638,2638_0
2,1,190,0.000000,1_190,1_190,190_1
3,2,78,0.804577,2_78,2_78,78_2
4,3,4,0.342788,3_4,3_4,4_3
5,3,79,0.800870,3_79,3_79,79_3
6,3,859,4.738878,3_859,3_859,859_3
7,4,3,3.723437,3_4,4_3,3_4
8,4,5,5.457894,4_5,4_5,5_4
9,4,16,0.143906,4_16,4_16,16_4


In [6]:
# summarize trips in each direction
ft_vol_sum = pd.DataFrame(bike_volume.groupby('ft_key')['bike_vol'].sum())
tf_vol_sum = pd.DataFrame(bike_volume.groupby('tf_key')['bike_vol'].sum())

ft_vol_sum.columns = ['ft_bvol']
tf_vol_sum.columns = ['tf_bvol']

tf_vol_sum.head()

,tf_bvol
tf_key,
0_2638,0.000000
0_732,0.000000
100000_100021,1.872970
100000_100364,12.577158
100000_99978,1.313319


In [7]:
# summarize trips in both directions
volume_sum = pd.DataFrame(bike_volume.groupby('key')['bike_vol'].sum())
volume_sum.columns = ['total_bvol']
volume_sum.head(10)

,total_bvol
key,
0_2638,0.000000
0_732,0.000000
100000_100021,15.077318
100000_100364,14.280019
100001_100002,0.000000
100002_100026,0.000000
100003_100004,0.000000
100004_100019,0.000000
100004_100036,0.000000


In [8]:
#Create FTkey and TF key to use for joining to bike volumes
links['key'] = np.where(links['from_node'].astype(int) < links['to_node'].astype(int), 
                              links['from_node'].astype(str) + "_"+ links['to_node'].astype(str), 
                              links['to_node'].astype(str) + "_"+ links['from_node'].astype(str))

links[['from_node', 'to_node','key']].head(10)

,from_node,to_node,key
0,3,859,3_859
1,4,858,4_858
2,4,3,3_4
3,5,4,4_5
4,6,5,5_6
5,7,6,6_7
6,8,7,7_8
7,9,5,5_9
8,10,9,9_10
9,10,6,6_10


In [9]:
# copy the links table
links2 = links[['link_id', 'key']].copy()

# join the links with the bike volumes using the common keys
link_bike_vol = links2.merge(volume_sum, left_on='key', right_on='key', how='left')
link_bike_vol2 = link_bike_vol.merge(ft_vol_sum, left_on='key', right_on='ft_key', how='left')
link_bike_vol3 = link_bike_vol2.merge(tf_vol_sum, left_on='key', right_on='tf_key', how='left')

# examine the results
print(links2.shape)
print(link_bike_vol3.shape)

(137281, 2)
(137281, 5)


In [10]:
link_bike_vol3.head(5)

,link_id,key,total_bvol,ft_bvol,tf_bvol
0,0,3_859,9.477756,4.738878,4.738878
1,1,4_858,0.000000,0.000000,0.000000
2,2,3_4,4.066225,0.342788,3.723437
3,3,4_5,5.705248,5.457894,0.247354
4,4,5_6,5.705248,5.457894,0.247354


In [11]:
# Examine the column names
links_shp.columns

Index(['Join_Count', 'TARGET_FID', 'FID_lines_', 'Name', 'Oneway', 'Speed',
       'AutoNetwor', 'BikeNetwor', 'PedNetwork', 'SourceData', 'DriveTime',
       'BikeTime', 'Pedestrian', 'Length_Mil', 'ConnectorN', 'RoadClass',
       'AADT', 'AADT_YR', 'BIKE_L', 'BIKE_R', 'Shape_Leng', 'id', 'Start_Key',
       'End_Key', 'Bike_Lane', 'Bike_Path', 'Bike_Blvd', 'SIGID', 'Signal',
       'BUFF_DIST', 'ORIG_FID', 'link_id', 'from_z', 'to_z', 'Slope_AB',
       'Slope_BA', 'Slope_Per', 'geometry'],
      dtype='object')

In [12]:
# export final result to csv
link_bike_vol3['link_id'] = link_bike_vol3['link_id'].astype('int64')

# join bike vol to links shapefile
links4 = links_shp.merge(link_bike_vol3, left_on='link_id', right_on='link_id', how='outer')

# export to shape
links4.to_file(r".\Results\links_bv.shp")


## Summarize zone trips by Attracting/Producing Zone

In [4]:
# read in zones
zones = gpd.read_file(r".\Data\microzones.shp")

### Read in trip tables, summarize, and format

In [5]:
def summarize_zones(trips_df, name):
    
    # summarize trips by attraction or production
    trips_sum_attr = pd.DataFrame(trips_df.groupby('azone')['bk'].sum())
    trips_sum_prod = pd.DataFrame(trips_df.groupby('pzone')['bk'].sum())
    
    # format tables
    trips_sum_attr['zone_id'] = trips_sum_attr.index
    trips_sum_attr.columns = [name + '_abk', 'zone_id']
    trips_sum_prod['zone_id'] = trips_sum_prod.index
    trips_sum_prod.columns = [name + '_pbk', 'zone_id']
    
    # join the attraction and production summary tables using zone id
    merged = trips_sum_attr.merge(trips_sum_prod, left_on='zone_id', right_on='zone_id', how='outer')
    return merged
    

In [6]:
# sch_univ = pd.read_csv(r".\Data\sch_univ_trip.csv")
# sch_univ_sum = summarize_zones(sch_univ, 'univ')
# sch_univ_sum.isnull().values.any()

In [7]:
# Discretionary trips (social trips, some recreation)
disc = pd.read_csv(r".\Data\disc_trip.csv")
disc_sum = summarize_zones(disc, 'disc')
del disc

# Maintenance trips (e.g. groceries)
maint = pd.read_csv(r".\Data\maint_trip.csv")
maint_sum = summarize_zones(maint, 'mnt')
del maint

# Maintenance trips non-home-based (e.g. groceries)
maint_nhb = pd.read_csv(r".\Data\maint_trip_nhb.csv")
maint_nhb_sum = summarize_zones(maint_nhb, 'mntnhb')
del maint_nhb

# Recreational family trips
rec_fam = pd.read_csv(r".\Data\rec_fam_trip.csv")
rec_fam_sum = summarize_zones(rec_fam, 'recfam')
del rec_fam

# Recreation long trips
rec_long = pd.read_csv(r".\Data\rec_long_trip.csv")
rec_long_sum = summarize_zones(rec_long, 'reclng')
del rec_long

# Recreation other trips (recreation that doesn't fall into family or long)
rec_oth = pd.read_csv(r".\Data\rec_oth_trip.csv")
rec_oth_sum = summarize_zones(rec_oth, 'recoth')
del rec_oth

# school (grade) trips
sch_grade = pd.read_csv(r".\Data\sch_grade_trip.csv")
sch_grade_sum = summarize_zones(sch_grade, 'grade')
del sch_grade

# school (university) trips
sch_univ = pd.read_csv(r".\Data\sch_univ_trip.csv")
sch_univ_sum = summarize_zones(sch_univ, 'univ')
del sch_univ

# Work trips
work = pd.read_csv(r".\Data\work_trip.csv")
work_sum = summarize_zones(work, 'wrk')
del work

# Work non-home-based trips
work_nhb = pd.read_csv(r".\Data\work_trip_nhb.csv")
work_nhb_sum = summarize_zones(work_nhb, 'wrknhb')
del work_nhb

In [8]:
rec_fam_sum

,recfam_abk,zone_id,recfam_pbk
0,0.010634,0,NaN
1,0.013320,1,NaN
2,0.008244,2,NaN
3,0.011433,3,NaN
4,0.007352,4,NaN
...,...,...,...
18807,0.639513,19212,1.435160
18808,0.333619,19213,1.634315
18809,0.831176,19214,2.474203
18810,1.356840,19215,0.009276


### Merge trip summarizes back to microzone shapefile

In [9]:
# Create a clean copy of zones dataset
zones2 = zones[['zone_id', 'CO_TAZID', 'TAZID', 'CO_FIPS', 'CO_NAME', 'geometry']].copy()
zones2['zone_id'] = zones2['zone_id'].astype('int64')

# Join trip tables
zones2 = zones2.merge(disc_sum, left_on='zone_id', right_on='zone_id', how='outer')
zones2 = zones2.merge(maint_sum, left_on='zone_id', right_on='zone_id', how='outer')
zones2 = zones2.merge(maint_nhb_sum, left_on='zone_id', right_on='zone_id', how='outer')
zones2 = zones2.merge(rec_fam_sum, left_on='zone_id', right_on='zone_id', how='outer')
zones2 = zones2.merge(rec_long_sum, left_on='zone_id', right_on='zone_id', how='outer')
zones2 = zones2.merge(rec_oth_sum, left_on='zone_id', right_on='zone_id', how='outer')
zones2 = zones2.merge(sch_grade_sum, left_on='zone_id', right_on='zone_id', how='outer')
zones2 = zones2.merge(sch_univ_sum, left_on='zone_id', right_on='zone_id', how='outer')
zones2 = zones2.merge(work_sum, left_on='zone_id', right_on='zone_id', how='outer')
zones2 = zones2.merge(work_nhb_sum, left_on='zone_id', right_on='zone_id', how='outer')

# preview table
zones2.head(30)

,zone_id,CO_TAZID,TAZID,CO_FIPS,CO_NAME,geometry,disc_abk,disc_pbk,mnt_abk,mnt_pbk,mntnhb_abk,mntnhb_pbk,recfam_abk,recfam_pbk,reclng_abk,reclng_pbk,recoth_abk,recoth_pbk,grade_abk,grade_pbk,univ_abk,univ_pbk,wrk_abk,wrk_pbk,wrknhb_abk,wrknhb_pbk
0,0,0,0,0,None,"POLYGON ((432271.913 4511646.351, 432587.351 4...",0.072392,NaN,0.174530,NaN,0.072954,NaN,0.010634,NaN,0.022836,NaN,0.066335,NaN,0.158398,NaN,0.371506,NaN,0.045686,NaN,0.034858,NaN
1,1,0,0,0,None,"POLYGON ((432587.351 4511685.482, 433708.825 4...",0.097771,NaN,0.224566,NaN,0.093868,NaN,0.013320,NaN,0.028422,NaN,0.079588,NaN,0.190242,NaN,0.771318,NaN,0.064802,NaN,0.049444,NaN
2,2,0,0,0,None,"POLYGON ((434783.825 4513368.123, 436523.175 4...",0.066655,NaN,0.147136,NaN,0.061503,NaN,0.008244,NaN,0.016771,NaN,0.041753,NaN,0.113639,NaN,0.655912,NaN,0.044216,NaN,0.033737,NaN
3,3,0,0,0,None,"POLYGON ((436523.175 4513990.279, 437808.876 4...",0.096555,NaN,0.214897,NaN,0.089827,NaN,0.011433,NaN,0.053676,NaN,0.064089,NaN,0.153230,NaN,1.121467,NaN,0.051686,NaN,0.039436,NaN
4,4,0,0,0,None,"POLYGON ((438170.076 4513697.879, 437971.005 4...",0.058983,NaN,0.134381,NaN,0.056171,NaN,0.007352,NaN,0.028617,NaN,0.047598,NaN,0.117019,NaN,0.531817,NaN,0.029747,NaN,0.022697,NaN
5,5,0,0,0,None,"POLYGON ((439107.670 4514080.595, 439893.846 4...",0.010444,NaN,0.024549,NaN,0.010262,NaN,0.001297,NaN,0.009402,NaN,0.028525,NaN,0.028460,NaN,0.019672,NaN,0.010192,NaN,0.007777,NaN
6,6,0,0,0,None,"POLYGON ((439893.846 4514550.662, 440268.999 4...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,0,0,0,None,"POLYGON ((441107.148 4514640.267, 442255.076 4...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,0,0,0,None,"POLYGON ((442255.076 4515303.123, 442409.608 4...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,0,0,0,None,"POLYGON ((443356.952 4517164.217, 442409.608 4...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
zones2.columns

Index(['zone_id', 'CO_TAZID', 'TAZID', 'CO_FIPS', 'CO_NAME', 'geometry',
       'disc_abk', 'disc_pbk', 'mnt_abk', 'mnt_pbk', 'mntnhb_abk',
       'mntnhb_pbk', 'recfam_abk', 'recfam_pbk', 'reclng_abk', 'reclng_pbk',
       'recoth_abk', 'recoth_pbk', 'grade_abk', 'grade_pbk', 'univ_abk',
       'univ_pbk', 'wrk_abk', 'wrk_pbk', 'wrknhb_abk', 'wrknhb_pbk'],
      dtype='object')

In [14]:
# fill na's with 0
zones3 = zones2.fillna(0)
zones3['total_abk'] = (zones3['disc_abk'] + zones3['mnt_abk'] + zones3['mntnhb_abk'] + 
                       zones3['recfam_abk'] + zones3['reclng_abk'] + zones3['recoth_abk'] + 
                       zones3['grade_abk'] + zones3['univ_abk'] + zones3['wrk_abk'] + zones3['wrknhb_abk']) 

zones3['total_pbk'] = (zones3['disc_pbk'] + zones3['mnt_pbk'] + zones3['mntnhb_pbk'] + 
                       zones3['recfam_pbk'] + zones3['reclng_pbk'] + zones3['recoth_pbk'] + 
                       zones3['grade_pbk'] + zones3['univ_pbk'] + zones3['wrk_pbk'] + zones3['wrknhb_pbk']) 

In [15]:
# then export to shape
zones3.to_file(r".\Results\Microzone_Trip_Summaries.shp")

## Merge zone attraction and production scores with the microzone geometry

In [53]:
# Create a clean copy of zones dataset
zones2 = zones[['zone_id', 'CO_TAZID', 'TAZID', 'CO_FIPS', 'CO_NAME', 'geometry']].copy()
zones2['zone_id'] = zones2['zone_id'].astype('int64')

ascore = pd.read_csv(r".\Data\zone_attraction_size.csv")
pscore = pd.read_csv(r".\Data\zone_production_size.csv")

zones3a = zones2.merge(ascore, left_on='zone_id', right_on='zone_id', how='outer')
zones3p = zones2.merge(pscore, left_on='zone_id', right_on='zone_id', how='outer')

In [54]:
# Fill NAs with -1, then export to shape
zones3a.fillna(-1).to_file(r".\Results\Microzone_A_Scores.shp")
zones3p.fillna(-1).to_file(r".\Results\Microzone_P_Scores.shp")

## Get Centroid Nodes

In [55]:
nodes = gpd.read_file(r".\Data\nodes.shp")
nodes['node_id'] = nodes.index
nodes.shape

(108655, 28)

In [56]:
nodes2 = nodes[['node_id', 'xcoord', 'ycoord', 'zcoord', 'geometry']].copy()
centroids = nodes2.merge(zones[['NODE_ID', 'zone_id']], left_on='node_id', right_on='NODE_ID', how='inner')
print(centroids.columns)

Index(['node_id', 'xcoord', 'ycoord', 'zcoord', 'geometry', 'NODE_ID',
       'zone_id'],
      dtype='object')


In [57]:
centroids = centroids[['node_id', 'xcoord', 'ycoord', 'zcoord', 'zone_id', 'geometry']].copy()
centroids.to_file(r".\Results\Microzone_Centroids.shp")